In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [15]:
import pandas as pd
import numpy as np

In [16]:
path_cleaned = "/content/drive/My Drive/Hackhathon 2020 Shared Drive/Dataset/SG_car_cleaned.csv"

In [17]:
data = pd.read_csv(path_cleaned)

In [18]:
data.head()

,Unnamed: 0,Unnamed: 0.1,trj_id,osname,pingtimestamp,latitude,longitude,speed,bearing,accuracy,timestamp,travel_time,pickup_time,dropoff_time,latitude_origin,longitude_origin,latitude_destination,longitude_destination,hour_of_day,day_of_week,is_weekend,is_weekday,is_wee_hours,is_rush_hours_morning,is_rush_hours_evening,sin_hour_of_day,cos_hour_of_day,sin_day_of_week,cos_day_of_week,haversine_km,haversine_m,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1
0,0,0,2,android,1554777528,1.270377,103.857875,25.623161,100,32.000,2019-04-09 02:38:48+00:00,1520.0,2019-04-09 02:38:48+00:00,2019-04-09 03:04:08+00:00,1.270377,103.857875,1.446609,103.806354,2,1,0,1,0,0,0,0.500000,0.866025,0.781831,0.623490,20.403093,12.677028,-0.025486,-0.076384,0.041837,0.094437
1,1,1134,3,ios,1555057936,1.282591,103.851082,0.000000,96,30.000,2019-04-12 08:32:16+00:00,1158.0,2019-04-12 08:32:16+00:00,2019-04-12 08:51:34+00:00,1.282591,103.851082,1.332855,103.977278,8,4,0,1,0,0,0,0.866025,-0.500000,-0.433884,-0.900969,15.091547,9.376812,-0.017611,-0.064838,-0.138717,-0.003314
2,2,2067,4,android,1555152160,1.355363,103.863157,5.960000,217,3.900,2019-04-13 10:42:40+00:00,1366.0,2019-04-13 10:42:40+00:00,2019-04-13 11:05:26+00:00,1.355363,103.863157,1.343218,103.858504,10,5,1,0,0,0,1,0.500000,-0.866025,-0.974928,-0.222521,1.445192,0.897939,-0.023023,0.008731,-0.019494,-0.003787
3,3,2946,5,ios,1555724348,1.340301,103.775742,4.760463,91,6.000,2019-04-20 01:39:08+00:00,1328.0,2019-04-20 01:39:08+00:00,2019-04-20 02:01:16+00:00,1.340301,103.775742,1.303071,103.900220,1,5,1,0,0,0,0,0.258819,0.965926,-0.974928,-0.222521,14.434456,8.968542,0.062661,-0.014213,-0.064685,-0.039977
4,4,4107,8,android,1554967474,1.333936,103.953225,15.250000,318,4.288,2019-04-11 07:24:34+00:00,1210.0,2019-04-11 07:24:34+00:00,2019-04-11 07:44:44+00:00,1.333936,103.953225,1.309601,103.833983,7,3,0,1,0,0,0,0.965926,-0.258819,0.433884,-0.900969,13.520439,8.400637,-0.114666,-0.004423,0.001871,-0.039493


In [19]:
data = data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'trj_id', 'osname', 'pingtimestamp', 'timestamp', 'latitude', 'longitude', 'speed', 'bearing', 'accuracy', 'pickup_time', 'dropoff_time', 'pickup_pca0', 'pickup_pca1', 'dropoff_pca0', 'dropoff_pca1'], axis=1)

In [20]:
data = data.drop(['is_weekend','is_weekday'], axis=1)

#Modelling

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import timeit

In [22]:
"""start = timeit.default_timer()
main()
end = timeit.default_timer()
print("Time taken in seconds:{}".format(end-start))"""

'start = timeit.default_timer()\nmain()\nend = timeit.default_timer()\nprint("Time taken in seconds:{}".format(end-start))'

## DATA

In [23]:
data = data.sample(frac=1).reset_index(drop=True)

train, test = train_test_split(data, test_size=0.5, train_size = 0.5)
validate, test = train_test_split(test, test_size = 0.3, train_size = 0.7)
print(len(train))
print(len(validate))
print(len(test))


13771
9639
4132


In [24]:
X_train = train.drop(['travel_time'], axis=1)
Y_train = train["travel_time"]
X_val = validate.drop(['travel_time'], axis=1)
Y_val = validate["travel_time"]
X_test = test.drop(['travel_time'], axis=1)
Y_test = test["travel_time"]

## Linear Regression

In [25]:
from sklearn.linear_model import LinearRegression

start = timeit.default_timer()
regressor = LinearRegression()  
regressor.fit(X_train, Y_train) #training the algorithm
end = timeit.default_timer()
print("Time taken in seconds:{}".format(end-start))

y_pred = regressor.predict(X_test)
mse = mean_squared_error(Y_test,y_pred)
print("RMSE: %.2f" % np.sqrt(mse))

Time taken in seconds:0.036615980000021864
RMSE: 230.20


In [26]:
from sklearn.linear_model import Lasso

start = timeit.default_timer()
clf = Lasso(alpha=0.1).fit(X_train,Y_train)
end = timeit.default_timer()
print("Time taken in seconds:{}".format(end-start))

pred = clf.predict(X_test)

mse = mean_squared_error(Y_test,pred)
print("RMSE: %.2f" % np.sqrt(mse))

Time taken in seconds:0.014184096999997564
RMSE: 230.27


In [27]:
from sklearn.linear_model import Ridge

start = timeit.default_timer()
rng = np.random.RandomState(0)
clf = Ridge(alpha=1.0)
clf.fit(X_train, Y_train)
end = timeit.default_timer()
print("Time taken in seconds:{}".format(end-start))

pred = clf.predict(X_test)

mse = mean_squared_error(Y_test,pred)
print("RMSE: %.2f" % np.sqrt(mse))

Time taken in seconds:0.008959916000037538
RMSE: 230.23


In [28]:
from sklearn.neural_network import MLPRegressor

start = timeit.default_timer()
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, Y_train)
end = timeit.default_timer()
print("Time taken in seconds:{}".format(end-start))

pred = regr.predict(X_test)

mse = mean_squared_error(Y_test,pred)
print("RMSE: %.2f" % np.sqrt(mse))

Time taken in seconds:40.408112388000006
RMSE: 221.57


In [29]:
from sklearn import tree
start = timeit.default_timer()
clf = tree.DecisionTreeClassifier().fit(X_train, Y_train)
end = timeit.default_timer()
print("Time taken in seconds:{}".format(end-start))

pred = clf.predict(X_test)

mse = mean_squared_error(Y_test,pred)
print("RMSE: %.2f" % np.sqrt(mse))

Time taken in seconds:5.547856221000018
RMSE: 302.59


In [30]:
from sklearn.ensemble import RandomForestClassifier

start = timeit.default_timer()
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, Y_train)
end = timeit.default_timer()
print("Time taken in seconds:{}".format(end-start))

pred = clf.predict(X_test)

mse = mean_squared_error(Y_test,pred)
print("RMSE: %.2f" % np.sqrt(mse))

Time taken in seconds:10.860441649999984
RMSE: 308.17


In [31]:
from sklearn import svm

start = timeit.default_timer()
regr = svm.SVR()
regr.fit(X_train, Y_train)
end = timeit.default_timer()
print("Time taken in seconds:{}".format(end-start))

pred = regr.predict(X_test)

mse = mean_squared_error(Y_test,pred)
print("RMSE: %.2f" % np.sqrt(mse))

Time taken in seconds:15.630175964999978
RMSE: 263.32
